# Limpieza y estructurado de datos del tiempo

Objetivo: unir todos los csv de datos del tiempo en uno solo. Estos datos ya están filtrados para la estación meteorológica del retiro, pero hay que filtrar los valores recogidos que nos interesan. El resultado se guarda en *./tiempo/retiro*.

In [ ]:
inPath = "../0 data/tiempo/"
outPath = "./tiempo/"

## Unión de datasets

In [ ]:
import pandas as pd

In [ ]:
# Leer los archivos json que empiezan con "tiempo" en la carpeta inPath
# y juntarlos en un solo DataFrame

import os
files = os.listdir(inPath)
files = [f for f in files if f.startswith("tiempo")]

dataframes = []
for f in files:
    dataframes.append(pd.read_json(inPath + f))

df = pd.concat(dataframes, ignore_index=True)
df.tail()

In [ ]:
# Inspeccionar los datos
df.info()
df.describe()

## Filtrado a columnas que nos interesan por estar también en la predicción

> Columnas: fecha, prec, tmin, tmax, dir, velmedia, racha

In [ ]:
# Quedarse con las columnas que nos interesan
# Columnas: fecha, prec, tmin, tmax, dir, velmedia, racha
df = df[["fecha", "prec", "tmin", "tmax", "dir", "velmedia", "racha"]]
df.info()

In [ ]:
df.tail()

## Limpieza y adaptación de los datos

 - Tipos numéricos

In [ ]:
# Cambiar Ip por 0 en la columna prec
df["prec"] = df["prec"].replace("Ip", 0)

# Cambiar 99 y 88 por NaN en la columna dir
df["dir"] = df["dir"].replace(99, None)
df["dir"] = df["dir"].replace(88, None)

# Cambiar comas por puntos en las columnas numéricas
df["prec"] = df["prec"].astype(str).str.replace(",", ".").astype(float)
df["tmin"] = df["tmin"].astype(str).str.replace(",", ".").astype(float)
df["tmax"] = df["tmax"].astype(str).str.replace(",", ".").astype(float)
df["velmedia"] = df["velmedia"].astype(str).str.replace(",", ".").astype(float)
df["racha"] = df["racha"].astype(str).str.replace(",", ".").astype(float)
df.info()


In [ ]:
df.tail()

 - Se observa que los datos de viento están incompletos (pero son importantes)
 Existe la posibilidad de emplear otra estación meteorológica (aeropuerto de Barajas pej)

 - También se encuentran datos incompletos en las temperaturas, aunque son pocos (se decide prolongar el día anterior), y en las precipitaciones, aunque son menos aún (se decide sustituir por 0)

In [ ]:
# Show 5 of november 2022
df[df["fecha"] == "2022-11-05"]

In [ ]:
# Prolongar temperatura del día anterior para los días que no tienen tmin o tmax

df["tmin"] = df["tmin"].fillna(method="ffill")
df["tmax"] = df["tmax"].fillna(method="ffill")

# Show 5 of november 2022
df[df["fecha"] == "2022-11-05"]

In [ ]:
# Show 11 of march 2022
df[df["fecha"] == "2022-03-11"]

In [ ]:
# Poner a 0 la precipitación de los días que no tienen
df["prec"] = df["prec"].fillna(0)

# Show 11 of march 2022
df[df["fecha"] == "2022-03-11"]

## Guardado de los resultados

In [ ]:
# Create directory if it doesn't exist
import os
if not os.path.exists(outPath):
    os.makedirs(outPath)

In [ ]:
# Save the DataFrame to a csv file
df.to_csv(outPath + "tiempo.csv", index=False, encoding="utf-8")